In [1]:
import xml.etree.ElementTree as ET
import json
import pandas as pd
from collections import defaultdict

In [2]:
#Hear was modified to 
#<?xml version="1.0" encoding="UTF-8"?>
#<TEI>
with open('british-diplomats-directory.xml') as fd:
    tree = ET.parse(fd)
root = tree.getroot()


In [3]:
# there is a problem preventing using findall from the root branch.
# getting the list of all sections
sections = defaultdict(list)
section_attrib = '{http://www.w3.org/XML/1998/namespace}id'
for child in root.iter():
    if section_attrib in child.attrib:
        sections[child.attrib[section_attrib]].append(child)

In [4]:
sections.keys()

dict_keys(['abbott-anthony-j', 'places', 'australia', 'perth', 'brazil', 'sao-paulo', 'montserrat', 'positions', 'consul-general', 'governor', 'order-of-chivalry', 'cmg', 'obe', 'section-a', 'section-a_a', 'section-a_b', 'section-a_c', 'section-a_d', 'section-a_e', 'section-a_f', 'section-a_g', 'section-a_h', 'section-a_i', 'section-a_j', 'section-a_k', 'section-a_l', 'section-a_m', 'section-a_n', 'section-a_o', 'section-a_p', 'section-a_q', 'section-a_r', 'section-a_s', 'section-a_t', 'section-a_u', 'section-a_w', 'section-a_y', 'section-a_z', 'section-b', 'section-c', 'section-d', 'section-e', 'section-e_a', 'section-e_b', 'section-e_c', 'section-e_d', 'section-e_e', 'section-e_f', 'section-e_g', 'section-e_h', 'section-e_i', 'section-e_j', 'section-e_k', 'section-e_l', 'section-e_m', 'section-e_n', 'section-e_o', 'section-e_p', 'section-e_q', 'section-e_r', 'section-e_s', 'section-e_t', 'section-e_u', 'section-e_v', 'section-e_w', 'section-e_y', 'section-f', 'section-g', 'section-h'

In [13]:
#for section A and E
section_e = list()
for post_section in [post for entry in sections["section-e"] for post in entry.findall('.//persName/../..')]:
    #name 
    for person in post_section.findall(".//persName/.."): #"the form tag"
        new_name = dict()
        new_name['persName'] = person.find("persName").text
        new_name['date'] = [a.text for a in person.findall("date")] #person.find("date").attrib.get("from")
        new_name['roleName'] = [a.text for a in person.findall("roleName")]    
    #seg
    new_name["roles"] = list()
    for seg in post_section.findall(".//seg"):
        new_role = dict()
        new_role["rs"] = seg.find("rs").text
        new_role["date"] = ", ".join(d.text for d in seg.findall("date"))
        
        new_name["roles"].append(new_role)
    section_e.append(new_name)


In [15]:
with open('section_e.json', 'w') as fp:
    json.dump(section_e, fp)

In [21]:
section_e_lines = list()
for line in section_e:
    new_line = dict()
    new_line["persName"] = line["persName"]
    new_line["date"] = ", ".join(line["date"])
    new_line["roleName"] = ", ".join(line["roleName"])
    for role in line["roles"]:
        section_e_lines.append({**new_line, **role})

In [ ]:
section_f_df = pd.DataFrame.from_records(section_e_lines)
section_f_df.index = section_f_df.index+1
section_f_df.fillna("")
section_f_df.to_csv("section_f.csv")

In [22]:
section_e_lines[2:5]

[{'persName': 'ABEL SMITH, Colonel SIR HENRY',
  'date': '1958-1966',
  'roleName': 'K.C.M.G., K.C.V.O., D.S.O.',
  'rs': 'Governor of Queensland'},
 {'persName': 'ABELL, SIR ANTHONY F.',
  'date': '1950-1959',
  'roleName': 'K.C.M.G.',
  'rs': 'Governor of Sarawak'},
 {'persName': 'ACHESON, ANDREW B.',
  'date': '1938-194_',
  'roleName': 'C.M.G.',
  'rs': 'Head of Pacific and Mediterranean Department, Colonial Office'}]

In [7]:
post_section.findall(".//seg")

[<Element 'seg' at 0x11778e728>, <Element 'seg' at 0x11778e818>]

In [8]:
person.find("date").attrib.get("from")

'1827'

# Reading Section B, C, D, F, G, H, I and J (Position with list of people), different from section A and E

In [5]:
#section_f and section_f
section_f = dict()

for post_section in [post for entry in sections["section-f"] for post in entry.findall('.//persName/../../..')]:
    post_title = post_section.find("head").text
    section_f[post_title] =list()
    for person in post_section.findall(".//persName/.."):
        new_post = dict()
        new_post['persName'] = person.find("persName").text
        new_post['date'] = [a.text for a in person.findall("date")]
        new_post['roleName'] = [a.text for a in person.findall("roleName")]
        section_f[post_title].append(new_post)


In [6]:
with open('section_f.json', 'w') as fp:
    json.dump(section_f, fp)

In [7]:
section_f_lines = list()
for key,value in section_f.items():
    head = key
    for persName in value:
        new_line = dict()
        new_line["persName"] = persName["persName"]
        new_line["date"] = ", ".join(persName["date"])
        new_line["roleName"] = ", ".join(persName["roleName"])
        new_line["Position"] = head
        section_f_lines.append(new_line)

In [8]:
section_f_df = pd.DataFrame.from_records(section_f_lines)
section_f_df.index = section_f_df.index+1
section_f_df.fillna("")
section_f_df.to_csv("section_f.csv")

In [9]:
section_f_df

,Position,date,persName,roleName
1,"ADEN (from 1937 until 1963; HIGH COMMISSIONER,...",1937-1940,Lieutenant-Colonel Sir Bernard R. Reilly,"K.C.M.G., C.I.E., O.B.E."
2,"ADEN (from 1937 until 1963; HIGH COMMISSIONER,...",1940-1944,Sir John H. Hall,"G.C.M.G., D.S.O., O.B.E., M.C."
3,"ADEN (from 1937 until 1963; HIGH COMMISSIONER,...",1944-1951,Sir Reginald S. Champion,"K.C.M.G., O.B.E."
4,"ADEN (from 1937 until 1963; HIGH COMMISSIONER,...",1951-1956,Sir Tom Hickinbotham,"K.C.M.G., K.C.V.O., C.I.E., O.B.E."
5,"ADEN (from 1937 until 1963; HIGH COMMISSIONER,...",1956-1960,Sir William H.T. Luce,"G.B.E., K.C.M.G."
6,"ADEN (from 1937 until 1963; HIGH COMMISSIONER,...",1960-1963,Sir Charles H. Johnston,G.C.M.G.
7,"ADEN (from 1937 until 1963; HIGH COMMISSIONER,...",1963-1965,Sir G. Kennedy N. Trevaskis,"K.C.M.G., O.B.E."
8,"ADEN (from 1937 until 1963; HIGH COMMISSIONER,...",1965-1967,Sir Richard G. Turnbull,G.C.M.G.
9,"ADEN (from 1937 until 1963; HIGH COMMISSIONER,...",1967,Lord Trevelyan,"K.G., G.C.M.G., C.I.E., O.B.E."
10,"ADEN (from 1937 until 1963; HIGH COMMISSIONER,...",1937-1940,Lieutenant-Colonel Morice C. Lake,C.M.G.


In [8]:
section_f["Assistant Under-Secretary of State (excluding Chief Clerks; up to 1945)"]

[{'persName': 'James Murray', 'date': ['1858-1869'], 'roleName': ['C.B.']},
 {'persName': 'Hon. C.W. Thomas Spring-Rice',
  'date': ['1869-1870'],
  'roleName': []},
 {'persName': 'Rt. Hon. Lord Ampthill',
  'date': ['1870-1871'],
  'roleName': ['G.C.B.', 'G.C.M.G.']},
 {'persName': 'Lord Tenterden', 'date': ['1871-1873'], 'roleName': ['K.C.B.']},
 {'persName': 'Sir T. Villiers Lister',
  'date': ['1873-1894'],
  'roleName': ['K.C.M.G.']},
 {'persName': 'Lord Pauncefote',
  'date': ['1876-1882'],
  'roleName': ['P.C.', 'G.C.B.', 'G.C.M.G.']},
 {'persName': 'Lord Currie', 'date': ['1882-1889'], 'roleName': ['G.C.M.G.']},
 {'persName': 'Lord Sanderson',
  'date': ['1889-1894'],
  'roleName': ['G.C.B.', 'K.C.M.G.']},
 {'persName': 'Sir H. Percy Anderson',
  'date': ['1894-1896'],
  'roleName': ['K.C.B.', 'K.C.M.G.']},
 {'persName': 'Viscount Bertie of Thame',
  'date': ['1894-1903'],
  'roleName': ['P.C.', 'G.C.B.', 'G.C.M.G.', 'G.C.V.O.']},
 {'persName': 'Rt. Hon. Sir Francis Hyde Villie